# Power System Planning Project

In [27]:
using DataFrames, CSV, Plots, JuMP, Statistics, HiGHS
import Dates
ENV["ROWS"]=120

120

## TMY Calculation

The following section shows how the hourly capacity factors were obtained for onshore wind, offshore wind and solar. By using the time series available at Open Power Systems Data for UK, the hourly capacity factors of these technologies are obtained. Then, for each of the technologies:
- We take the monthly average of the capacity factor, capturing the long term tendency of the values.
- For each month, we take the average of each year of the available data. If there is an hour from that month, in that year, that has no data, then that year is not considered for the analysis for the month in particular.
    - The average of each year is computed, so, for each month, we have a list with the average capacity factor for each year during that month.
- We compare the long term tendency (average year) with the average of each month. The month with the lowest deviation from the average is chosen, and that month is the reference month for the typical year.

This allows to get a year created from the available years that capture the averga behaviour, but taking into account variability throughout the days, as opposed of using an average value for capacity factors.

_Open Power System Data. 2020. Data Package Time series. Version 2020-10-06. [https://doi.org/10.25832/time_series/2020-10-06](https://doi.org/10.25832/time_series/2020-10-06) (Primary data from various sources, for a complete list see URL)._
    

In [2]:
timeseries = CSV.read("Data/time_series_60min_singleindex.csv", DataFrame)
columns = names(timeseries)
to_keep = columns[occursin.("GB",columns) .& occursin.("profile",columns) .& (occursin.("wind",columns) .| occursin.("solar",columns))]
to_keep_ts = vcat(columns[1], to_keep)

timeseries = timeseries[!,to_keep_ts]
CSV.write("Data/timeseries_uk_erv.csv", timeseries)

#We'll create a TMY with the Solar data, Onshore wind data and offshore data
timeseries[!,"timestamp"] = Dates.DateTime.(timeseries[!,"utc_timestamp"],"yyyy-mm-ddTHH:MM:SSZ");
# We filter the data so we don't have missing values anywhere
timeseries_dict = Dict()
#for column in to_keep
#    timeseries_dict[column] = timeseries[.!ismissing.(timeseries[!,column]),["timestamp",column]]
#end
for column in to_keep
    timeseries_dict[column] = timeseries[!,["timestamp",column]]
    #timeseries_dict[column] = timeseries_dict[column][.!ismissing.(timeseries_dict[column][!,column]),:]
end

# From each date, we save the year, month, day and hour, as the data is hourly
for column in to_keep
    timeseries_dict[column][!,"year"] = Dates.year.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"month"] = Dates.month.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"day"] = Dates.day.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"hour"] = Dates.hour.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"missing"] = ismissing.(timeseries_dict[column][!,column])
end

long_term_data = Dict()
long_term_year = Dict()
# Now, we group by month and day so we can get the average capacity factor
for element in timeseries_dict
    name, df = element
    df = element[2];
    gdf = groupby(df, [:month, :year])
    gdf = combine(gdf, name .=> mean => name)
    long_term_data[name] = gdf

    df = gdf;
    gdf = groupby(df, :month)
    gdf = combine(gdf, name .=> mean => name)
    long_term_year[name] = gdf
end

#upper, lower = max(long_term_data["GB_GBN_solar_profile"][!,:year]...),min(long_term_data["GB_GBN_solar_profile"][!,:year]...)
#upper, lower = 2018, 2015
#years = upper-lower;


UndefVarError: UndefVarError: `Dates` not defined

In [3]:
u = timeseries_dict["GB_GBN_solar_profile"]

UndefVarError: UndefVarError: `timeseries_dict` not defined

In [4]:
monthly_avg = Dict()
for element in timeseries_dict
    name, df = element
    avgs = []
    valid_years_list =[]
    for k in 1:12
        # Determine month with missing data
        monthly_df = groupby(df[df.month.==k,:],:year)
        monthly_df = combine(monthly_df, "missing" .=> sum => "missing")
        valid_years = monthly_df[monthly_df[!,"missing"] .== 0,:year]
        avg = mean(df[(df.month.==k) .& (df.year .∈ Ref(valid_years)),name])
        push!(avgs,avg)
        push!(valid_years_list,valid_years)
    end
    monthly_avg[name] = avgs
    monthly_avg[name*"_years"] = valid_years_list
end

avg_data = Dict()
for element in timeseries_dict
    name, df = element
    avg_matrix = []
    for k in 1:12
        monthly_df = groupby(df[df.month.==k,:],:year)
        monthly_df = combine(monthly_df, "missing" .=> sum => "missing")
        valid_years = monthly_df[monthly_df[!,"missing"] .== 0,:year]
        yearly_means = []
        for j in valid_years
            yearly_mean = mean(df[(df.month.==k) .& (df.year.==j),name])
            push!(yearly_means,yearly_mean)
        end
        push!(avg_matrix,yearly_means)
    end
    avg_data[name] = avg_matrix
end


UndefVarError: UndefVarError: `timeseries_dict` not defined

In [5]:
monthly_avg["GB_GBN_solar_profile_years"]

KeyError: KeyError: key "GB_GBN_solar_profile_years" not found

In [6]:
TMYS = Dict()
for element in avg_data
    name, matrix = element
    best_value = monthly_avg[name]
    df = timeseries_dict[name][!,[name, "month", "year","day","hour"]]
    TMY = DataFrame([[],[],[],[],[]], [name,"year","month","day","hour"])
    best_years = []
    years_lookup = monthly_avg[name*"_years"]
    for k in 1:12
        error = abs.(matrix[k].-best_value[k])
        best_year_index = findall(x->x==min(error...), error)
        best_year = years_lookup[k][best_year_index[1]]
        best_year_df = df[(df.month.==k) .& (df.year.==best_year),[name,"year","month","day","hour"]]
        TMY = vcat(TMY,best_year_df)
        push!(best_years,best_year)
    end
    # Set every timestamp to 2023
    TMY[!,"timestamp"] = Dates.DateTime.("2023".*"-".*string.(TMY[!,"month"]).*"-".*string.(TMY[!,"day"]).*"T".*string.(TMY[!,"hour"]).*":00:00","yyyy-mm-ddTHH:MM:SS");
    TMYS[name] = TMY[!,["timestamp",name,"month"]]
    #CSV.write("TMY_"*name*".csv", TMY)
end



UndefVarError: UndefVarError: `avg_data` not defined

In [7]:
toCSVdict = []
for element in TMYS
    name, df = element
    push!(toCSVdict,df[!,["timestamp",name]])
end

innerjoin(toCSVdict..., on=:timestamp) |> CSV.write("Data/TMY_uk_erv.csv")

MethodError: MethodError: no method matching innerjoin(; on::Symbol)

Closest candidates are:
  innerjoin(!Matched::AbstractDataFrame, !Matched::AbstractDataFrame; on, makeunique, validate, renamecols, matchmissing, order)
   @ DataFrames C:\Users\nacho\.julia\packages\DataFrames\58MUJ\src\join\composer.jl:756
  innerjoin(!Matched::AbstractDataFrame, !Matched::AbstractDataFrame, !Matched::AbstractDataFrame...; on, makeunique, validate, matchmissing, order)
   @ DataFrames C:\Users\nacho\.julia\packages\DataFrames\58MUJ\src\join\composer.jl:773


In [8]:
plots = []
for element in long_term_year
    name, df = element
    df_tmy = TMYS[name]
    df_raw = timeseries_dict[name]
    df_tmy = groupby(df_tmy, :month)
    df_tmy = combine(df_tmy, name .=> mean => name)
    display_parts = titlecase.(split(name,"_"));
    display_name = join(display_parts[3:end]," ");
    plot(df[!,:month],df[!,name],label="Long Term",color="orange",legend=:topleft, title=display_name)
    for k in lower:upper
        df_year = df_raw[df_raw.year.==k,:]
        df_year = groupby(df_year, :month)
        df_year = combine(df_year, name .=> mean => name)
        plot!(df_year[!,:month],df_year[!,name],label="", color="blue", alpha=0.25)
    end
    p=plot!(df_tmy[!,:month],df_tmy[!,name],label="TMY", color="green")
    push!(plots,p)
end
plot(plots..., layout=(4,1), size=(1000,1000), yformatter=y->string(trunc(Int,100*y))*"%", show=true)
xticks!(1:12,["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])
#savefig("Results/TMY_comparison.png")

UndefVarError: UndefVarError: `long_term_year` not defined

In [9]:
groupby(TMYS["GB_GBN_solar_profile"],:month)

KeyError: KeyError: key "GB_GBN_solar_profile" not found

## Variable Definitions and Model

### Model 0: Basecase
#### Sets
- $G$: set of available technologies
- $H$: set of hours to be simulated
#### Parameters
- $cf_{i,t}$: capacity factor of generator $g\in G$ in period $t \in H$ [p.u.]
- $CAPEX_g$: annuitized investment costs of generator $g\in G$ [$\$/\text{MW-y}$]
- $OM^F_{g}$: annuitized fixed operation and maintenance costs of generator $g \in G$ [$\$/\text{MW-y}$]
- $OM^V_{g}$: variable operation and maintenance cost of generator $g \in G$ [$\$/\text{MWh}$]
- $HR_{g}$: heat rate of generator $g \in G$ [$\text{MWh}/\text{MMBTU}$]
- $FC_{g}$: fuel cost of generator $g \in G$ [$\$/\text{MMBTU}$]
- $D_{t}$: electricity demand during hour $t \in H$ [$\text{MWh}$]
- $VoLL$: value of lost load. Payment done for each MWh of unserved energy [$\$/\text{MWh}$]
#### Variables
- $GEN_{g,t}$: generation of generator $g \in G$ in period $t \in H$
- $CAP_{g}$: installed capacity of generator $g \in G$
- $NSE_{t}$: energy non-served in period $t\in H$

#### Constraints
- Demand balance: the energy generated plus the non-served demand must be equal to the demand at all time.
$$\sum_{g \in G} GEN_{g,t}+NSE_{t} = D_t \quad \forall \, t\in H$$
- Capacity limits: the energy generated by each generator cannot exceed its installed capacity, times the capacity factor.
$$GEN_{g,t} \leq CAP_{g}\cdot cf_{g,t} \quad \forall \, t\in H,\,g \in G$$

#### Objective Function
The objective of the problem is to minimise the operation, investment and unserved energy throughout the period

$$\underset{CAP, GEN, NSE}{\min} \underbrace{\sum_{g \in G} \left(CAPEX_g+OM^F_g\right)CAP_g}_{\text{Investment Costs}} + \underbrace{\sum_{g \in G}\sum_{t \in H}\left(FC_g\cdot HR_g + OM^V_g\right)GEN_{g,t}}_{\text{Operation Costs}} + \underbrace{\sum_{t \in H}VoLL\cdot NSE_t}_{\text{Unserved Energy Costs}}$$

In [10]:
generators = CSV.read("Data/CostsData.csv",DataFrame);
G = generators.Gen;

# Separate the generators into renewable and conventional
G_erv = G[occursin.("Wind",G) .| occursin.("Solar",G)];
G_conv = G[G.∉ Ref(G_erv)];

# We want to select the columns that do work for us. We will remove those in kW
columns = names(generators)
to_use = columns[.!occursin.("KW",columns) .& .!occursin.("Gen",columns)]

gen_data = Dict()
for column in to_use
    colname = split(column)[1]
    gen_data[colname] = JuMP.Containers.DenseAxisArray(generators[!,column],G)
end

In [11]:
demand = CSV.read("Data/demanddata_Hourly.csv",DataFrame);
D = demand.ND;

In [12]:
H = 1:length(D)

1:8760

In [13]:
cap_df = CSV.read("Data/TMY_uk_erv.csv",DataFrame);
names_x = names(cap_df);
for gen in G_erv
    if occursin("Solar",gen)
        tech = "solar"
    else
        tech = lowercase(split(gen,"W")[1])
    end
    for name in names_x
        if occursin(tech,name)
            rename!(cap_df, name => gen)
            names_x = names(cap_df)
        end
    end

end

cap_df = cap_df[!,G_erv];
capacity_factors = Dict()
for column in names(cap_df)
   capacity_factors[column] = JuMP.Containers.DenseAxisArray(cap_df[!,column],H)
end

VoLL = 1000;

In [14]:
FirstExpansionModel = Model(Gurobi.Optimizer)
@variables(FirstExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
end);

@constraints(FirstExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
end)

@objective(FirstExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H))

UndefVarError: UndefVarError: `Gurobi` not defined

In [15]:
maximum(D)

87809

In [16]:
optimize!(FirstExpansionModel)

UndefVarError: UndefVarError: `FirstExpansionModel` not defined

In [17]:
generation = value.(GEN).data*ones(8760,1)
total_generation = JuMP.Containers.DenseAxisArray(generation[:],G)
total_co2 = sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*total_generation[g] for g in G)

UndefVarError: UndefVarError: `GEN` not defined

In [18]:
SecondExpansionModel = Model(Gurobi.Optimizer)
@variables(SecondExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
end);

@constraints(SecondExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
    CO2Constraint, sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for g in G, h in H) <= total_co2 * 0.5
end)

@objective(SecondExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H))

UndefVarError: UndefVarError: `Gurobi` not defined

In [19]:
optimize!(SecondExpansionModel)

UndefVarError: UndefVarError: `SecondExpansionModel` not defined

In [20]:
display(value.(CAP))
CO2price = abs(dual.(CO2Constraint));

UndefVarError: UndefVarError: `CAP` not defined

In [21]:
ThirdExpansionModel = Model(Gurobi.Optimizer)
@variables(ThirdExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
    CO2[g in G] >= 0
end);

@constraints(ThirdExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
    #CO2Constraint, sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for g in G, h in H) <= total_co2 * 0.5
    CO2Emissions[g in G], CO2[g] >= sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for h in H)
end)

@objective(ThirdExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H)
+ sum(CO2price*CO2[g] for g in G))

UndefVarError: UndefVarError: `Gurobi` not defined

In [22]:
optimize!(ThirdExpansionModel)

UndefVarError: UndefVarError: `ThirdExpansionModel` not defined

In [23]:
value.(CAP)

UndefVarError: UndefVarError: `CAP` not defined